##Downloading data from Telegram

In [ ]:
from asyncio import run
from telethon import TelegramClient, events, sync, functions, types, errors
import pandas as pd
import datetime as dt

In [ ]:
#We need to get api_id and api_hash, you can do that here: https://my.telegram.org
API_ID = 0
API_HASH = ''

Telethon is based on the built-in asyncio library, so all of its methods are asynchronous. Keep this in mind when using it.
The documentation has a section devoted to this: https://docs.telethon.dev/en/stable/concepts/asyncio.html

In [ ]:
async def main():
    # Authorization
    client = TelegramClient('me_session', API_ID, API_HASH,
                            lang_code='en', system_lang_code='en')
    await client.start()

    # Getting channel profiles
    result = await client(functions.channels.GetChannelsRequest(id=['@newyorktimes', '@science', '@memes']))
    # Convert query result to a list of dictionaries
    chnls = [c.to_dict() for c in result.chats]
    # Create a dataframe and write it to CSV
    df_channels = pd.DataFrame(chnls)
    df_channels.to_csv('tlgrm_channels.csv', sep=';')

    # Downloading messages
    # Specify the date from which we want to receive messages. Parameter reverse=False changes the date to "till what day".
    min_date = dt.datetime(2022, 4, 22, 0, 0, 0)
    # Getting the first 10 messages since April 22, 2022, in chronological order
    result = await client.get_messages('@science', max_id=0, min_id=0, reverse=True, offset_date=min_date, limit=10)
    # Do the same conversion procedure as with the channels
    msgs = [msg.to_dict() for msg in result]
    df_messages = pd.DataFrame(msgs)
    df_messages.to_csv('tlgrm_messages.csv', sep=';')

    # Disconnecting
    await client.disconnect()


run(main())